In [36]:
import pandas as pd
import pickle,re 
import itertools as it
from __future__ import print_function

In [2]:
regles="Longitudinal-09-T100000-F14858-X-Morphomes-Regles.pkl"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
debug=False

In [3]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: print (minLongueur, maxLongueur, file=logfile)
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug: print (phonemes, file=logfile)
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print (forme1,forme2,patron, file=logfile)
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print (patron12,patron21, file=logfile)
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print ("patron1", patron, file=logfile)
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print ("patron2", patron, file=logfile)
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

In [4]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n, file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                print (forme, file=logfile)
                print ("pas de classe",idClasseForme, file=logfile)
                print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            if debug:
                print (forme, file=logfile) 
                print ("pas de patron", file=logfile)
        return sortieForme
        

In [5]:
with open(rep+regles, 'rb') as input:
    classesFinales = pickle.load(input)

## Définition des cases à étudier

In [34]:
sampleCases=set(i for (i,o) in classesFinales)
#minCases=list(sampleCases)
minCases=["ii1P", "pP", "ii1S", "ai3S", 
          "ai1S", "pi2S", "inf", 
          "pi2P", "ppFP", "ps1S", "pI2S", 
          "pI2P", "ps2P", 
          "pi1S", "pi1P", "pi3P", 
          "fi1S", "pI1P"]
#'"'+'", "'.join(minCases)+'"'
cfCases=["ii1S", "pi2S", "inf", "ppFP", "ps1S", "ps2P", "pi1P", "pi3P", "fi1S"]
len(cfCases)

9

In [141]:
for (e,s) in it.combinations(cfCases,2):
    print (e,s)
    for patron in classesFinales[(e,s)].entree:
        print (patron,classesFinales[(e,s)].patrons[patron])
        print (classesFinales[(e,s)].entree[patron],classesFinales[(e,s)].sortie[patron])

ii1S pi2S
avE-E ^savE$
avE E
EjE-wa ^asEjE$
EjE wa
wE- ^(.*[ptkbdgfsSvzZmnJlr])uwE$
wE 
6.E-9. ^(.*[ptkbdgmnJl])6([jlr])E$
6(.)E 9\g<1>
vE- ^(.*[jriEe6a])vE$
vE 
jE- ^(.*[ptkbdgfsSvzZjlrwH][iyEe926auOo])jE$
jE 
E- ^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJjlr])E$
E 
tE- ^(.*[jrwHiyEe926auOoêûâô])tE$
tE 
yvE-wa ^byvE$
yvE wa
aJE-ê ^(.*)ZwaJE$
aJE ê
6nE-jê ^(.*[ptbdfsvz])6nE$
6nE jê
uvE-2 ^puvE$
uvE 2
u.E-9. ^murE$
u(.)E 9\g<1>
O.E-o. ^(.*)O([ptbdfsvz])E$
O(.)E o\g<1>
sE- ^(.*[ptkbdgfsSvzZmnJjlrwHE96aOêûâô][iEe])sE$
sE 
6.E-2. ^kr6zE$
6(.)E 2\g<1>
6vE-wa ^(.*[tdsz])6vE$
6vE wa
6nE-â ^(.*)pr6nE$
6nE â
pE- ^êtErôpE$
pE 
dE- ^(.*[ptbdfsvzrE96O][r6Oâô])dE$
dE 
jE-i ^(.*[ptbdfsvzmnr])jE$
jE i
EjE-je ^asEjE$
EjE je
ulE-2 ^vulE$
ulE 2
HE- ^(.*[ptbdmnl])yHE$
HE 
6zE-E ^(.*)f6zE$
6zE E
6.E-E. ^(.*[ptkbdgfsSvzZmnJlr])6([ptbdfsvzmnl])E$
6(.)E E\g<1>
6vE-2 ^pl6vE$
6vE 2
mE- ^(.*)dOrmE$
mE 
zE- ^(.*[ptbdfsvzjlH][iEe])zE$
zE 
EJE-ê ^(.*[ptbdfsvzlr])EJE$
EJE ê
EE-e ^krEE$
EE e
alE-va ^al

6(.)ir 9\g<1>
6..e-9.. ^av6gle$
6(.)(.)e 9\g<1>\g<2>
6vwar-wav ^(.*[tdsz])6vwar$
6vwar wav
âdr-En ^(.*)prâdr$
âdr En
êdr-aJ ^r6Zwêdr$
êdr aJ
Er-ô ^(.*)fEr$
Er ô
e- ^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJjlr])e$
e 
avwar-ô ^avwar$
avwar ô
tr-s ^(.*[mnr])Etr$
tr s
r-v ^(.*[pkbg][rw][iEea])r$
r v
r-z ^(.*[ptbdfsvzjlH][iEe])r$
r z
ale-vô ^ale$
ale vô
O.e-o. ^(.*)O([ptkbdgfsSvzZmnJlr])e$
O(.)e o\g<1>
He- ^(.*[ptkbdgmnJl])yHe$
He 
u.ir-9. ^murir$
u(.)ir 9\g<1>
êdr-EJ ^(.*[ptbdfsvzlr])êdr$
êdr EJ
we- ^(.*[ptkbdgfsSvzZ])uwe$
we 
Etr-sô ^Etr$
Etr sô
6.e-9. ^(.*[bdgvzZmnJlr])6([mnlr])e$
6(.)e 9\g<1>
Erir-jEr ^akErir$
Erir jEr
ir- ^(.*[ptkbdgfsSvzZmnJjrwHiyEe926auOoêûâô][fvjrwHiyEe926auOoêûâô][ptbdfsvzmnr])ir$
ir 
r- ^(.*[ptkbdgfsSvzZmnJjrwHiyEe926uOoêûô][jrwHiyEe926auOoêûâô][ptbdfsvzjiEea])r$
r 
udr-Olv ^rEzudr$
udr Olv
je- ^(.*[ptkbdgfsSvzZjlrwH][iEea])je$
je 
inf fi1S
-E ^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZjriEe6a])r$
 E
6.e-E.6rE ^(.*[ptkbdgfsSvzZmnJr])6([t

In [105]:
r1=ur"^(.*[ptkbdgfsSvzZjlrwH][iyEe926auOo])jE$"
r2=ur"^(.*[jrwHiyEe926auOoêûâô])tE$"
r3=ur"^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJjlr])E$"

In [113]:
def getPositionsRegex(regex):
    result=[]
    regex=re.sub(ur"[()]","",regex.replace(".*","X"))
    chunks=[c for c in re.split(ur"(\[[^\]]+\])",regex) if c!=""]
    if chunks[0].startswith("^"): chunks[0]=chunks[0][1:]
    if chunks[-1].endswith("$"): chunks[-1]=chunks[-1][:-1]
    for chunk in chunks:
        if chunk.startswith("["): 
            result.append(chunk.replace("[","").replace("]",""))
        else:
            result.extend(chunk)
    return result

In [219]:
def getIntersectionRegex(p1,p2,debug=False):
    temp=[]
    p1.reverse()
    p2.reverse()
    for i in range(min(len(p1),len(p2))):
        temp.append(set(p1[i])&set(p2[i]))
    temp.reverse()
    result=[]
    for c in temp:
        result.append("".join(c))
    if "" in result:
        result=""
    return result

In [227]:
def transformeExp(positions,patron):
    temp=positions[:]
    e0,s0=patron.split("-")
    e1=[e for e in re.split(ur"(\.)",e0) if e!=""]
    e1.reverse()
    s1=[e for e in re.split(ur"(\.)",s0) if e!=""]
    s1.reverse()
    print (e1,s1)
    temp.reverse()
    result=[]
    for n,p in enumerate(temp):
        if n<len(e1):
            if e1[n] in p:
                if n<len(s1):
                    s=s1[n][::-1]
                    result.extend(s)
            elif e1[n]==".":
                result.append(p)
            else:
                print ("wrong pattern")
                result=[]
                break
        else:
            result.append(p)
    result.reverse()
    return result
        

In [229]:
transformeExp([u'X', u'O', u'ptbdfsvz', u'E'], "E-")

['E'] []


[u'X', u'O', u'ptbdfsvz']

In [230]:
transformeExp(getPositionsRegex(ur"^(.*)O([ptbdfsvz])E$"),u"E-")

[u'E'] []


[u'X', u'O', u'ptbdfsvz']

In [56]:
paireInit=(cfCases[0],cfCases[1])
contextes={}
contextes[paireInit]=[classesFinales[paireInit].patrons[c] for c in classesFinales[paireInit].patrons]
for s in cfCases[1:]:
    print (e,s)
    for tranformation in classesFinales[(e,s)].patrons:
        contexte=classesFinales[(e,s)].patrons[tranformation]
        print (tranformation,contexte)
        print (re.sub(ur"\[[^\]]+\]",".",contexte).replace(".*","X").replace("(","").replace(")",""))

ii1S pi2S
avE-E ^savE$
^savE$
EjE-wa ^asEjE$
^asEjE$
wE- ^(.*[ptkbdgfsSvzZmnJlr])uwE$
^X.uwE$
6.E-9. ^(.*[ptkbdgmnJl])6([jlr])E$
^X.6.E$
vE- ^(.*[jriEe6a])vE$
^X.vE$
jE- ^(.*[ptkbdgfsSvzZjlrwH][iyEe926auOo])jE$
^X..jE$
E- ^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJjlr])E$
^X..E$
tE- ^(.*[jrwHiyEe926auOoêûâô])tE$
^X.tE$
yvE-wa ^byvE$
^byvE$
aJE-ê ^(.*)ZwaJE$
^XZwaJE$
6nE-jê ^(.*[ptbdfsvz])6nE$
^X.6nE$
uvE-2 ^puvE$
^puvE$
u.E-9. ^murE$
^murE$
O.E-o. ^(.*)O([ptbdfsvz])E$
^XO.E$
sE- ^(.*[ptkbdgfsSvzZmnJjlrwHE96aOêûâô][iEe])sE$
^X..sE$
6.E-2. ^kr6zE$
^kr6zE$
6vE-wa ^(.*[tdsz])6vE$
^X.6vE$
6nE-â ^(.*)pr6nE$
^Xpr6nE$
pE- ^êtErôpE$
^êtErôpE$
dE- ^(.*[ptbdfsvzrE96O][r6Oâô])dE$
^X..dE$
jE-i ^(.*[ptbdfsvzmnr])jE$
^X.jE$
EjE-je ^asEjE$
^asEjE$
ulE-2 ^vulE$
^vulE$
HE- ^(.*[ptbdmnl])yHE$
^X.yHE$
6zE-E ^(.*)f6zE$
^Xf6zE$
6.E-E. ^(.*[ptkbdgfsSvzZmnJlr])6([ptbdfsvzmnl])E$
^X.6.E$
6vE-2 ^pl6vE$
^pl6vE$
mE- ^(.*)dOrmE$
^XdOrmE$
zE- ^(.*[ptbdfsvzjlH][iEe])zE$
^X..zE$
EJE-ê ^(.*[ptbdfsvzlr])E

### Relations entre cases par nombre de règles

In [35]:
longueur=0
nbPaires=0
print ("longueur=%d"%longueur)
for paire,valeur in sorted(classesFinales.items(),key=lambda x: len(x[1].entree.keys())):
    transformations=valeur.entree.keys()
    if len(transformations)!=longueur:
        print ("%d paires pour %d transformation(s)"%(nbPaires,longueur))
        longueur=len(transformations)
        nbPaires=0
        print ()
        print ("longueur=%d"%longueur)
    else:
        nbPaires+=1
    print ("%s => %s :"%paire, valeur.entree.keys(), len(valeur.classe))
print ("%d paires pour %d transformation(s)"%(nbPaires,longueur))

longueur=0
0 paires pour 0 transformation(s)

longueur=1
ai3P => ai1S : [u'r-'] 1
ppFP => ppFP : ['-'] 1
ai3P => ai3P : ['-'] 1
ps2P => ps2P : ['-'] 1
inf => inf : ['-'] 1
ii2P => ii2P : ['-'] 1
pc2P => pc2P : ['-'] 1
pI1P => pI1P : ['-'] 1
ii2P => ii1P : [u'e-\xf4'] 1
ai1P => ai3S : [u'm-'] 1
pi1P => pi1P : ['-'] 1
is3P => ai1P : [u's-m'] 1
pP => pP : ['-'] 1
pi2P => pi2P : ['-'] 1
is3P => is3P : ['-'] 1
ii1S => ii1S : ['-'] 1
ai1P => ai1P : ['-'] 1
ai1S => ai3P : [u'-r'] 1
ai3S => ai1P : [u'-m'] 1
is3P => ai3S : [u's-'] 1
ppMP => ppMP : ['-'] 1
ps2S => ps2S : ['-'] 1
ii1P => ii1P : ['-'] 1
pc1P => pc1P : ['-'] 1
ai3S => ai3S : ['-'] 1
ps1S => ps1S : ['-'] 1
ii1P => ii2P : [u'\xf4-e'] 1
ai2P => ai2P : ['-'] 1
ai3S => is3P : [u'-s'] 1
ai1P => is3P : [u'm-s'] 1
pI2S => pI2S : ['-'] 1
ai1S => ai1S : ['-'] 1
31 paires pour 1 transformation(s)

longueur=2
ai1P => ai1S : [u'm-', u'am-E'] 2
fi2P => pc2P : [u'e-je', u'e-ije'] 2
fi1P => pc1P : [u'\xf4-j\xf4', u'\xf4-i\xf4'] 2
fi1S => pc1P : [u

ppFP => ai3S : [u'z-', u't-vi', u'O.t-u.y', u'e-a', u't-zi', u'6ny-\xea', u'\xeat-aJi', u'Hisj\xf4-y', '-', u'swaj\xf4-fy', u't-', u'\xeat-EJi', u'e-aki', u'y-i', u'Et-i', u'Ert-ri'] 16
31 paires pour 16 transformation(s)

longueur=17
ps2S => fi1S : [u'as-6rE', u'-6rE', u'wa-ErE', u'wa-6rE', u'9.-u.rE', u'-irE', u'aS-OrE', u'His-urE', u'En-\xe2drE', u'z-rE', u'9j-udrE', u'En-\xeadrE', u'-rE', u'aj-irE', u'E-OrE', u'wa-jErE', u's-trE'] 19
ppFP => pI2P : [u'y-Ese', u'y-waje', u'i-e', u'y-wase', '-', u'y-Eje', u'y-6ve', u'iz-6ne', u'iz-Ete', u'y-e', u't-ve', u'iz-Eje', u'Ert-re', u'\xeat-EJe', u't-ze', u'y-aSe', u'\xf4-e'] 18
fi1S => ps2S : [u'irE-aj', u'jErE-wa', u'6rE-wa', u'u.rE-9.', u'udrE-9j', u'\xe2drE-En', u'urE-His', u'OrE-E', u'6rE-as', u'irE-', u'\xeadrE-En', u'ErE-wa', u'trE-s', u'rE-', u'6rE-', u'rE-z', u'OrE-aS'] 18
pi2S => ps2S : [u'-d', u'E-as', u'-s', u'\xe2-En', u'va-aj', u'a-E', '-', u'E-aS', u'je-wa', u'2-His', u'-t', u'-v', u'2-9j', u'-m', u'E-swa', u'\xea-En', u'-z'] 